In [16]:
import torch
from torchvision import transforms
from torch.utils.data import DataLoader

from sklearn.model_selection import train_test_split

from nn import *
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path

Prepare encoder

In [4]:
autoencoder = Autoencoder()
autoencoder.load_state_dict(torch.load("./train_output/20230722-141814/best_model.pth"))

encoder_state_dict = {}
for key in autoencoder.state_dict().keys():
    if key.startswith("encoder"):
        encoder_state_dict[key] = autoencoder.state_dict()[key]

encoder = Encoder()
encoder.load_state_dict(encoder_state_dict)

<All keys matched successfully>

Create NN

In [46]:
encoder_with_knn = EncoderWithKNN(encoder, num_classes=10, k=20)

Data preparation

In [8]:
class DigitDataset():
    def __init__(
        self, data_path: Path, transformse: bool = False
    ) -> None:
        super().__init__()
        self.dataset = pd.read_csv(data_path)
        self.transforms = transforms.ToTensor()

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        data = self.dataset.iloc[index]
        label, data = data[0], data[1:]
        data = np.reshape(np.array(data, dtype=np.uint8), (28,28))
        data = self.transforms(data) # Transform values to be in range 0-1

        return data, label

In [9]:
train_path = Path("./Dataset/train.csv")
train_dataset = DigitDataset(data_path=train_path)
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=False)

Extract features from the training dataset


In [13]:
# Extract features from the training dataset
all_features = []
all_labels = []
for images, labels in tqdm(train_loader):
    with torch.no_grad():
        features = encoder_with_knn(images).numpy()
        all_features.append(features)
        all_labels.append(labels.numpy())
all_features = np.concatenate(all_features, axis=0)
all_labels = np.concatenate(all_labels, axis=0)

100%|██████████| 1313/1313 [00:05<00:00, 251.70it/s]


In [14]:
all_features.shape

(42000, 64, 1, 1)

In [15]:
all_labels.shape

(42000,)

Split data into training and testing sets

In [17]:
X_train, X_test, y_train, y_test = train_test_split(all_features, all_labels, test_size=0.1, random_state=42)

In [18]:
print(f"X train shape: {X_train.shape}")
print(f"y train shape: {y_train.shape}")

print(f"X test shape: {X_test.shape}")
print(f"y test shape: {y_test.shape}")

X train shape: (37800, 64, 1, 1)
y train shape: (37800,)
X test shape: (4200, 64, 1, 1)
y test shape: (4200,)


Fix the shape. <br>
From # N, 64, 1, 1 ---> # N, 64

In [21]:
X_train = np.squeeze(X_train)
X_test = np.squeeze(X_test) 
y_train = np.squeeze(y_train) 
y_test = np.squeeze(y_test)

print(f"X train shape: {X_train.shape}")
print(f"y train shape: {y_train.shape}")

print(f"X test shape: {X_test.shape}")
print(f"y test shape: {y_test.shape}")

X train shape: (37800, 64)
y train shape: (37800,)
X test shape: (4200, 64)
y test shape: (4200,)


Train the KNN classifier on the training set


In [47]:
encoder_with_knn.fit_knn(X_train, y_train)

Test the KNN classifier on the testing set

In [48]:
predictions = encoder_with_knn.predict_knn(X_test)

Calculate test accuracy

K = 3

In [25]:
accuracy = np.mean(predictions == y_test)
print("Test Accuracy:", accuracy)

Test Accuracy: 0.9773809523809524


K = 4

In [29]:
accuracy = np.mean(predictions == y_test)
print("Test Accuracy:", accuracy)

Test Accuracy: 0.9761904761904762


K = 5

In [33]:
accuracy = np.mean(predictions == y_test)
print("Test Accuracy:", accuracy)

Test Accuracy: 0.9752380952380952


K = 2

In [37]:
accuracy = np.mean(predictions == y_test)
print("Test Accuracy:", accuracy)

Test Accuracy: 0.9719047619047619


K = 6

In [41]:
accuracy = np.mean(predictions == y_test)
print("Test Accuracy:", accuracy)

Test Accuracy: 0.975


K = 10

In [45]:
accuracy = np.mean(predictions == y_test)
print("Test Accuracy:", accuracy)

Test Accuracy: 0.9716666666666667


K = 20

In [49]:
accuracy = np.mean(predictions == y_test)
print("Test Accuracy:", accuracy)

Test Accuracy: 0.9666666666666667


Not good results. 
Try adding dimensionality in  encoder. from 64 --> 64, 2(3)
Try adding transforms on data:
 - Add to Encoder training?
 - Add to knn training, create multiple loaders